# Lec 26-28: Functional Programming Paradigm: Basic Concepts

Jianwen Zhu <jzhu@eecg.toronto.edu>
v2.0, 2024-09

## Agenda

- Control Flow & Side Effect
- Closures
- Currying
- High-Order Functions

## Control Flow & Side Effects

What is functional programming (FP), anyway? What are the common characteristics of programming languages with cool names such as Lisp, Scheme, Haskell, ML, OCAML, Clean, Mercury, or Erlang (or a few others)?

* Functions as first class citizen: That is, everything you can do with "data" can be done with
functions themselves (such as passing a function to another function), and there are "high order" functions
that are functions operating on functions.

* Recursion as primary control structure: In some languages, no other "loop" construct exists.

* List as primary value type: Lists are often used with recursion on sub-lists as a substitute for loops.

* No state, or no side effect in "Pure" functional languages: This excludes the almost ubiquitous pattern in imperative languages of assigning first one, then another value to the same variable to track the program state.

* No statements: FP either discourages or outright disallows statements, and instead works with the evaluation
of expressions (in other words, functions plus arguments). In the pure case, one program is one expression (plus supporting definitions).


You have already seen some of the FP features in Python! Examples include: 

1. Sequences and List comprehension;
2. High-order functions such as map(), reduce(), filter()
3. Recursions

To better understand FP, we first try to understand why FP can do *WITHOUT* 
1. control statement;
2. side effect (variable assignment);

which seem to be so ubiqutous in imperative programming, and can express everything as expression (pun intended)!


### Eliminating If-Then-Else Statements

Consider the following statements: 

```
if <cond1>: func1()
elif <cond2>: func2()
else: func3()
```

We want to rewrite it with an equivalent *short-circuit expression*, 

```
(<cond1> and func1()) or (<cond2> and func2()) or (func3())
```

Note that evaluation semantics of this expression. In particular, func1() is called only when <cond1> is true; and <cond2> is evaluated only if <cond1> is evaluated to be false, and so on. 

In [ ]:
x = 3

def pr(s) : return s

(x == 1 and pr('one')) or (x == 2 and pr('two')) or (x == 3 and pr('three')) 

### Anonymous Function with Lambda expression

In fact, in FP, a function itself is an expression, and you do not have to define a function with a name. 
Let's rewrite above with Lambda expression, which evaluates to a function value. The lambda expression starts with a keyword lambda, followed by a list of arguments before the colon, after which it is an expression representing the return value. 

Just a side story that early computing pioneer builds the theoretical foundation of computing using a theory called "lambda calulus", which is a functional language. This is where the name of lambda came from.

In [ ]:
pr = lambda s: s
namenum = lambda x: (x==1 and pr("one")) \
    or (x==2 and pr("two")) \
    or (pr("other"))

In [ ]:
namenum(1)

In [ ]:
namenum(2)

In [ ]:
namenum(3)

### Built-in High Order Functions: map(), reduce(), filter()

Lambda function shows the the first class status of functions in FP. This can be combined
nicely with the "operator" concept: function on functions, or simply high-order-functions. 
We already learned a few builtin high-order functions: 

* map() performs the passed function on each corresponding item in the specified list(s), and
returns a list of results.

* reduce() performs the passed function on each subsequent item and an internal accumulator
of a final result; 

* filter() uses the passed function to "evaluate" each item in a list, and return a winnowed list
of the items that pass the function test.

NOTE: Since Python 3.0 these builtin functions has been moved to the functools package.

In [ ]:
from functools import *

reduce(lambda n,m:n*m, range(1,10))

### Eliminating For Loop

If we see: 

```
for e in lst: func(e) # statement-based loop
```

We can easily replace it with: 
```
list(map( func, lst ))
```
or list comprehension:

```
[ func(e) for e in lst ] 
```


### Eliminating Statement Sequence

What about a sequence of statements like the following? 

```
print( 'i am statement1' )
print( 'i am statement2' )
print( 'i am statement3' )
```

Suppose each statement can be representated by a function as follows, we can use list comprehension to evaluate them in sequence.

In [ ]:
f1 = lambda: print( 'i am statement1' )
f2 = lambda: print( 'i am statement2' )
f3 = lambda: print( 'i am statement3' )

In [ ]:
# let's create an function that "applies" a function
do_it = lambda f: f()

[do_it(f) for f in [f1, f2, f3]]

### Eliminating While Loop

Give the imperative while loop statement: 

```
# statement-based while loop
while <cond>:
    <pre-suite>
    if <break_condition>:
        break
    else:
        <suite>
```

We can convert into FP using recursion: 

```
 def while_block():
     <pre-suite>
     if <break_condition>:
         return 1
     else:
          <suite>
    return 0

while_FP = lambda: (<cond> and while_block()) or while_FP()
while_FP()
```

Note the use of short-circuit expression. 

Now let's look at a concrete example:

In [ ]:
# imperative version of "echo()"

def echo_IMP():
    while 1:
        x = input("IMP -- ")
        if x == 'quit':
            break
        else: 
            print( x )
            
echo_IMP()


Let's convert into FP:

In [ ]:
# utility function for "identity with side-effect"
def monadic_print(x):
    print( x )
    return x

# FP version of "echo()"
echo_FP = lambda: monadic_print(input("FP -- "))=='quit' or echo_FP()
echo_FP()

What we have accomplished is that we have managed to express a little program that involves I/O, looping, and conditional statements as a pure expression with recursion (in fact, as a function object that can be passed elsewhere if desired). We do still utilize the utility function monadic_print(), but this function is completely general, and can be reused in every functional program expression we might create later (it's a one-time cost). Notice that any expression containing monadic_print(x)evaluates to the same thing as if it had simply contained x. FP
(particularly Haskell) has the notion of a "monad" for a function that "does nothing, and has a sideeffect in the process."

### Eliminating Side Effect

We now knows how to convert:

1. A (block) sequence of statements;
2. An if-then-else statement;
3. A for loop statement;
4. A while loop statements;

into FP, all with expressions. We can pretty much write the entire porgram using an expression, and accomplish a task without updating a single variable, that is, a program *without* side-effects!

Let's look at one concrete example below. We will start with an imperative code, and then get rid of it using FP.  The goal here is to print out a list of pairs of numbers whose product is more than 25. The numbers that make up the pairs are themselves taken from two other lists. 

An imperative approach to the goal might look like:

In [ ]:
xs = [1,2,3,4]
ys = [10,15,3,22]
bigmuls = []

for x in xs:
    for y in ys:
        if x*y > 25:
            bigmuls.append((x,y))

print( bigmuls )

A functional approach to our goal eliminates side-effects (and potential errors that goes with them) altogether. 

We start by defining a function that duplicate each element of a list by certain number of times.

In [ ]:
from functools import *

dupelms = lambda lst,n: reduce(lambda s,t:s+t, map(lambda l,n=n: [l]*n, lst))

In [ ]:
dupelms([1,2,3], 2)

We can then easily define a function that returns the Cartesian set of two list: the set of all pairs from elements of the two list.

In [ ]:
combine =  lambda xs,ys: zip( xs*len(ys), dupelms(ys,len(xs)) )

In [ ]:
list(combine( [1,2,3], [5,6] ))

Note that after Python 3, we have to use list() for what's returned by zip(). It is a bit annoying but it is for a good reason of efficiency. More about this later. But we could alternatively use list comprehension.

In [ ]:
[x for x in combine( (1,2,3), (5,6) )]

We were just one step away as we know that list comprehension can add filters. 

In [ ]:
bigmuls = lambda xs, ys : [(x,y) for (x,y) in combine(xs,ys) if x*y > 25]

bigmuls( [1,2,3,4], [10,15,3,22] )

Note we introduced dupelems, combine as intermediate expression to enhance readiblity, but we could have written everything as a single expression, in one line!

## Closure

So what is a closure, anyway?  A closure, like an object instance, is a way of carrying around a bundle of data and functionality, wrapped up together. If you have ever used a callback function in C, and remembers the pain of wrapping and passing data for the callback, you will know why closure is a much more elegant replacement. 

A *nested function* is a function defined in the lexical scope of another function. Let's look at the following nested function lazy_pow, which is returned by its parent function. 

NOTE: Does it look familiar to you? Does decorator rings the bell?

In [ ]:
def pow_later(x): 
    y = 2 
    def lazy_pow(): 
        print('calculate pow({}, {})...'.format(x, y)) 
        return pow(x, y) 
    return lazy_pow

In [ ]:
my_pow = pow_later(3)

In [ ]:
my_pow

In [ ]:
my_pow()

Obviously, the variable y and the parameter x are local variables of pow_later function. So when my_pow() was called, the pow_later function had already returned, and its local variables also had gone. But in fact my_pow() still remembered the vaules of x and y even the outer scope pow_later was long gone. How did this happen?

### Free Variable

If a variable in a function is neither a local variable nor a parameter of that function, this variable is called a *free variable* of that function. In short, free variables are variables that are used locally, but defined in an enclosing scope. In our case, x is a parameter of pow_later and y is a local variable of pow_later. But within lazy_pow, x and y are free variables.

### What is Closure

Generally speaking, a closure is a structure (code blocks, function object, callable object, etc.) storing a function together with an *environment*. The environment here means information about free variables that function bounded, especially values or storage locations of free variables.

Specifically speaking, my_pow, actually the function object returned by calling pow_later(x), is a closure. Note that the closure for lazy_pow extends the scope of lazy_pow function to include the binding for the free variables: x and y.

For example, a closure is created, returned and assigned to my_pow after following function call. >>> my_pow = pow_later(3) Essentially, this closure is the codes of function lazy_pow together with free variables x and y.



### Inspecting Closure

We are going through some backdoors below: However, you are NOT supposed to do this during actual coding, we are doing it just to help understanding.

In [ ]:
my_pow.__code__

In [ ]:
my_pow.__code__.co_freevars

Meanwhile, pow_later will also keep names of local variables that are referenced by its nested functions in co_cellvars attribute of its code object.

In [ ]:
pow_later.__code__.co_cellvars

However, where is the values of free variables?

In [ ]:
dir(my_pow)

In [ ]:
my_pow.__closure__

Note that my_pow has an attribute named __closure__ and it's a tuple with two elements.

In [ ]:
dir(my_pow.__closure__[0])

In [ ]:
my_pow.__closure__[0].cell_contents

In [ ]:
my_pow.__closure__[1].cell_contents

So __closure__ is a tuple of cells that contain bounded values of free variables.

### What's NOT a closure

Functions without free variables are not closures.

In [ ]:
def f(x): 
    def g(): 
        pass
    return g

In [ ]:
h = f(1)

In [ ]:
h

In [ ]:
h.__code__.co_freevars

In [ ]:
print( h.__closure__ )

Global variables are not free variables in Python. So global functions are not closures.

In [ ]:
data = 200 #global
def d() : #global
    print( data )

In [ ]:
d.__code__.co_freevars

In [ ]:
print( d.__closure__ )

### Non-Local

Suppose we want to change the behavior of above example slightly, as shown below:

In [ ]:
def pow_later(x): 
    y = 2 
    def lazy_pow(): 
        print('calculate pow({}, {})...'.format(x, y))
        result = pow(x, y) 
        y = y + 1 # increase y return result return lazy_pow
        return result
    return lazy_pow

In [ ]:
my_pow = pow_later(3)
my_pow

So far so good, but: 

In [ ]:
my_pow()

The error message is clear enough. It's a UnboundLocalError.
y is a local variable: because it is assigned inside lazy_pow, Python treated it as a local variable instead of a free variable, but then reading y yields an undefined value since it is never initialized.

To deal with this situation, a nonlocal declaration was introduced in Python 3. It marks a variable as a free variable even though it is assigned a new value within the function. So the correct version should be:


In [ ]:
def pow_later(x): 
    y = 2 
    def lazy_pow(): 
        nonlocal y # nonlocal declaration 
        print('calculate pow({}, {})...'.format(x, y)) 
        result = pow(x, y) 
        y = y + 1 
        return result 
    return lazy_pow

In [ ]:
my_pow = pow_later(3)

In [ ]:
my_pow()

In [ ]:
my_pow()

In [ ]:
my_pow()

## Currying

We now learn another important concept called currying. It is named after Haskell Curry, the Mathematitian deeply connected to Lambda calculus, the foundation of functional programming. 

### Function Specialization

Let's start by the need we often encounter in practice to specialize a general function. The reason for specialization can be:

1. Performance optimization;
2. Cognitive load reduction.


In [1]:
def multiply(a, b):
    return a*b

Now suppose we want to specialize multiple() into double().

In [2]:
def double(a):
    return multiply(a,2)

result = double(10)
print(result)  

20


Obviously, double() is simpler (has smaller cognitive load, so to speak) as you only need to deal with a single parameter.

What if we want to have a general way of function specialization. Our friend functools comes to rescue.

The following shows the syntax of the partial function from the functools module:

```
functools.partial(fn, *args, **kwargs)
```

The partial function returns new partial object, which is a callable. When you call the partial object, Python calls the fn function with the positional arguments args and keyword arguments kwargs.

The following example shows how to use the partial function to define the double function from the multiply function:

In [3]:
from functools import partial

def multiply(a, b):
    return a*b


double = partial(multiply, b=2)

result = double(10)
print(result)

20


This is how it works.

1. import the partial function from the functools module.
2. define the multiply function.
3. return a partial object from the partial function and assign it to the double variable. When you call the double, Python calls the multiply function where b argument defaults to 2.

If you pass more arguments to a partial object, Python appends them to the args argument. When you pass additional keyword arguments to a partial object, Python extends and overrides the kwargs arguments.

Therefore, it’s possible to call the double like this:

```
double(10, b=3)
```

In this example, Python will call the multiply function where the value of the b argument is 3, not 2.

Roughly speaking, the behavior of partial is equivalent to the following: 

```
def partial(func, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = keywords.copy()
        newkeywords.update(fkeywords)
        return func(*(args + fargs), **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc
```

It basically “freezes” some portion of a function’s arguments and/or keywords resulting in a new object with a simplified signature.    

### Currying with partial() 

Currying is a functional design pattern primarily used to reduce function with *multiple* arguments to *a chain of functions* that takes one argument each. In fact, in some "pure" functional program, a function only takes *ONE* argument, and currying is the primary means to implement what is equivalent to multiple argument function. This make such functional programming language (in fact, the original lambda calculus) extremely elegant. 

We can implement currying with partial() specializatin scheme just introduced (it can be considered a special case).

In [5]:
def mult(x, y, z):
  return x * y * z

In [6]:
from functools import partial
 
mult_10 = partial(mult, 10)
mult_10_20 = partial(mult_10, 20)
print(mult_10_20(30))

6000


You can see that each of mult_10(), mult_10_20() is a function that takes only one argument, where mult_10() is mult() specialized with 10 as x, and mult_10_20() is mult() specialized with 10 as x, and 20 as y.

In a "pure" functional programming language with "native" syntax, function application (calling a function) does not need "()": so 
our familiar syntax
```
func(argument)
```
will simply be:
```
func argment
```

Using the example above, mult(10,20,30) will simply be written as:
```
mult 10 20 30
```

But what it actually means is that:

```
mult(10)(20)(30)
```


### Currying with a decorator

Currying can be implemented much more efficiently by using the decorator. Recall that a decorator wraps code or functionality around a function in order to enhance what the function does. 


In [8]:
from inspect import signature

def curry(func):
 
  def inner(arg):
 
    #checking if the function has one argument,
    #then return function as it is
    if len(signature(func).parameters) == 1:
      return func(arg)
 
    return curry(partial(func, arg))       ## Note the recursion here!
 
  return inner

In [9]:
@curry
def mult(x, y, z):
  return x * y * z

In [10]:
print(mult(10)(20)(30))

6000


### Tax Calculation Example

In [11]:
@curry
def taxcalc( income, rate, deduct ) :
    return (income - deduct) * rate

In [12]:
print( "tax due = %d" % taxcalc(50000)( 0.30 )( 10000 ) )

tax due = 12000


It obviously works, but stare at it carefully: we were NOT passing multiple argument to the function taxcalc, but applying one argument, at a time, that is, currying! Please simulate in your head what Python was doing there.

### A Note on Lambda Calculus

Lambda calculus consists of constructing lambda terms and performing reduction operations on them. In the simplest form of lambda calculus, terms are built using only the following rules:

1. $x$: A variable is a character or string representing a parameter.
2. $\lambda x.M $: A lambda abstraction is a function definition, taking as input the bound variable $x$
 and returning the body $M$
3. $(M\ N)$: An application, applying a function $M$ to an argument $N$, Both $M$ and $N$ are lambda terms.

The reduction operation is simply: $(\lambda x . M N) → M [x := N]$ replacing the bound variables with the argument expression in the body of the abstraction.

Amazingly, a language as simply as above is proven equivalent to a Turinging machine!


## Recap

The most distincitive feature of functional programming is that it contains no state and side effects (state updating actions). As a result, a function program is just a giant expression. The "assignment" you see in a function program is just a *binding* of an expression value to a name for readibility rather than a variable (state) update. We learned:

1. how any imperative program can be transformed into a functional program;
2. why functions are first-class citenzen and how to use an lambda expression;≠
3. how high-order functions are defined and used;
4. how closure (functions with their environment) are defined and used;
5. how currying can eliminate functions with multiple arguments.

All of these make a functional programming language:
1. Simple with minimal core;
2. Safe with no possibility of memory error.

